In [1]:
!pip install --quiet pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 92.6 MB/s eta 0:00:00


# PCA+QNN

In [2]:
import pennylane as qml
import pandas as pd
from pennylane import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

/usr/local/lib/python3.11/dist-packages/pennylane/capture/capture_operators.py:33: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.4.28. You have version 0.5.2 installed. Please downgrade JAX to <=0.4.28 to avoid runtime errors.
  warnings.warn(


In [3]:
df=pd.read_csv("/content/qpca_based_reduced_features_with_labels_darwin.csv")
df.head()

,Feature1,Feature2,Label
0,3.028078,2.610559,1
1,6.664082,4.747633,1
2,2.848981,2.719746,1
3,6.557455,2.384333,1
4,3.996637,2.778244,1


In [4]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()


In [5]:
X_q=df.drop("Label",axis=1)
y_q=df["Label"]

In [6]:
y_q = label_encoder.fit_transform(y_q)

In [7]:
scaler = StandardScaler()
X_q = scaler.fit_transform(X_q)

In [8]:
# Split into training and testing sets
X_train_q, X_test_q, y_train_q, y_test_q = train_test_split(X_q, y_q, test_size=0.2, random_state=42)

In [9]:
# Quantum device
n_qubits = 2
dev = qml.device("default.qubit", wires=n_qubits)

In [10]:
# Quantum model
def quantum_circuit(inputs, weights):
    # Encoding the inputs into the quantum state
    for i in range(n_qubits):
        qml.RY(inputs[i], wires=i)

    # Variational circuit with weights
    for i in range(n_qubits):
        qml.RY(weights[i], wires=i)
        qml.RZ(weights[i + n_qubits], wires=i)
    qml.CNOT(wires=[0, 1])
    for i in range(n_qubits):
        qml.RY(weights[i + 2 * n_qubits], wires=i)

In [11]:
# Define the QNode
@qml.qnode(dev)
def qnode(inputs, weights):
    quantum_circuit(inputs, weights)
    return qml.expval(qml.PauliZ(0))

In [12]:
def binary_cross_entropy(predictions, targets):
    predictions = np.clip(predictions, 1e-9, 1 - 1e-9)  # Avoid log(0)
    targets = np.array(targets)  # Convert targets to NumPy array
    return -np.mean(targets * np.log(predictions) + (1 - targets) * np.log(1 - predictions))


In [13]:
# Define the hybrid model
def predict(X, weights):
    predictions = np.array([qnode(x, weights) for x in X])
    return (predictions + 1) / 2  # Rescale to [0, 1]

In [26]:
# Initialize weights
weights = np.random.uniform(-np.pi, np.pi, size=(3 * n_qubits,))

# Training the model
opt = qml.GradientDescentOptimizer(stepsize=0.01)
epochs = 20

for epoch in range(epochs):
    for x, y_true in zip(X_train_q, y_train_q):
        weights = opt.step(lambda w: binary_cross_entropy(predict([x], w), [y_true]), weights)

    # Calculate loss for this epoch
    train_predictions = predict(X_train_q, weights)
    train_loss = binary_cross_entropy(train_predictions, y_train_q)
    print(f"Epoch {epoch + 1}: Loss = {train_loss}")


Epoch 1: Loss = 0.611299605099388
Epoch 2: Loss = 0.6026706957439277
Epoch 3: Loss = 0.6002910150340238
Epoch 4: Loss = 0.599445470846353
Epoch 5: Loss = 0.5990740482780055
Epoch 6: Loss = 0.5988694652413507
Epoch 7: Loss = 0.5987324205927478
Epoch 8: Loss = 0.5986265657427382
Epoch 9: Loss = 0.5985372202486852
Epoch 10: Loss = 0.5984582070734628
Epoch 11: Loss = 0.5983869968485109
Epoch 12: Loss = 0.5983226719501159
Epoch 13: Loss = 0.5982649948397117
Epoch 14: Loss = 0.598213960007587
Epoch 15: Loss = 0.5981695756436782
Epoch 16: Loss = 0.5981317620185617
Epoch 17: Loss = 0.5981003129299965
Epoch 18: Loss = 0.5980748928405953
Epoch 19: Loss = 0.598055054275494
Epoch 20: Loss = 0.5980402657443359


In [27]:
# Test the model
test_predictions = predict(X_test_q, weights)
test_predictions_binary = (test_predictions > 0.5).astype(int)
accuracy = np.mean(test_predictions_binary == y_test_q)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 74.29%


In [28]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
y_pred=test_predictions_binary
print("Classification Report:")
print(classification_report(y_test_q, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.65      0.87      0.74        15
           0       0.65      0.87      0.74        15
           1       0.87      0.65      0.74        20
           1       0.87      0.65      0.74        20

    accuracy                           0.74        70
   macro avg       0.76      0.76      0.74        70
weighted avg       0.77      0.74      0.74        70



In [29]:
from sklearn.metrics import roc_auc_score

auc = roc_auc_score(y_test_q,y_pred)
print(f"AUC: {auc}")

AUC: 0.7583333333333333
